# Run analysis on multiple files

In [13]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing
import h5py

In [15]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FinerResolution_BendingStiffness'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-23'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-18'
# data_folder = '/home/deepak/Dropbox/LacryModeling/ModellingResults/StochasticActivity/NormalDistribution'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_750'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FilamentLength_Sweeps/activity_time_750'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-28'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-10-19'

data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrength_sweep'

print(os.listdir(data_folder))

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.' and 'analysis' not in fileNames):
            files_list.append(os.path.join(dirs,fileNames))
#         if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
#             files_list.append(os.path.join(dirs,fileNames))

print('No:of Simulation files: ', len(files_list))

def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)


    root_folder, file_name = os.path.split(file)
    
    # @@@ Better to use Metadata from the HDF5 file so there is no possible delinking of data and metadata
    df = pd.DataFrame({'N particles':[filament.Np],'radius':[filament.radius],'bond length':[filament.b0],
                       'spring constant':[filament.k], 'kappa_hat':[filament.kappa_hat], 
                       'force strength':[filament.F0], 'stresslet strength':[filament.S0], 
                       'potDipole strength':[filament.D0], 'simulation type':[filament.sim_type], 
                       'boundary condition 0':[filament.bc[0]], 'boundary condition 1':[filament.bc[-1]], 
                       ' activity time scale':[filament.activity_timescale], 'viscosity':[filament.mu]})
    
    periodic_flag = False 
    min_period = None
    threshold_index = 0  # Activity cycle when the dynamics transitions to a periodic cycle
    
    filament.filament_tip_coverage(save = True)
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) == int(filament.activity_timescale*500)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period, threshold_index = filament.classify_filament_dynamics()
    else:
        simulation_completed = False
        
    activity_cycles_completed = int(filament.Time[-1]/filament.activity_timescale)
    
    if(min_period is None):
        # if dynamics is aperiodic
        
        df['period'] = float(np.nan)
        df['threshold index'] = 0
    else:
        df['period'] = min_period
        df['threshold index'] = threshold_index
        
        
    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    df['activity cycles completed'] = activity_cycles_completed
    
    print('Activity cycles completed: {}'.format(activity_cycles_completed))
    
    # Calculate the filament tip angles (at constant phase) and save them to a file
    
    phase = 0
    angles = filament.compute_tip_angle_at_constant_phase(phase_value = phase)
    print(angles)
    
    

    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
    df.to_csv(os.path.join(save_folder, save_file))
    
    # Create a dataset and save the data
    save_file = file_name[:-5] + '_analysis.hdf5'


    with h5py.File(os.path.join(save_folder, save_file), "w") as f:
        dset = f.create_group("analysis data")
        dset.attrs['N particles'] = filament.Np
        dset.attrs['radius'] = filament.radius
        dset.attrs['bond length'] = filament.b0
        dset.attrs['spring constant'] = filament.k
        dset.attrs['kappa_hat'] = filament.kappa_hat
        dset.attrs['force strength'] = filament.F0
        dset.attrs['stresslet strength'] = filament.S0
        dset.attrs['potDipole strength'] = filament.D0
        dset.attrs['simulation type'] = filament.sim_type
        dset.attrs['activity time scale'] = filament.activity_timescale
        dset.attrs['viscosity'] = filament.mu
        dset.attrs['boundary condition 0'] = filament.bc[0]
        dset.attrs['boundary condition 1'] = filament.bc[-1]
        dset.attrs['Sim complete'] = simulation_completed
        dset.attrs['activity cycles completed'] = activity_cycles_completed
        dset.attrs['threshold cycle'] = df['threshold index']
        
#         print(periodic_flag)
#         print(min_period)
        dset.attrs['periodic'] = df['periodic dynamics']
        dset.attrs['period'] =  df['period']
        
        dset.attrs['constant phase'] = phase
        dset.create_dataset("Tip angles", data = angles)
        
    
    

['2021-08-28', '2021-10-19', '2021-10-21', '2021-10-20', '2021-08-20']
No:of Simulation files:  620


In [16]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))



  0%|          | 0/620 [00:00<?, ?it/s][Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.

  2%|▏         | 12/620 [00:00<00:07, 81.75it/s][Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.8s

  4%|▍         | 24/620 [00:01<00:50, 11.89it/s][Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    2.9s

  6%|▌         | 36/620 [00:03<00:55, 10.49it/s][Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    6.1s

  8%|▊         | 48/620 [00:06<01:38,  5.81it/s][Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    6.6s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:   18.9s

 10%|▉         | 60/620 [00:18<04:30,  2.07it/s][Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:   37.4s

 12%|█▏        | 72/620 [00:37<07:41,  1.19it/s][Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:   40.1s

 15%|█▌        | 96/620 [00:47<05:34,  1.57it/s][Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:   47.9s

 17%|█▋        | 108/620 [00:52<04:45,  1.7

In [21]:
print(np.linspace(0.5,3,40))

[0.5        0.56410256 0.62820513 0.69230769 0.75641026 0.82051282
 0.88461538 0.94871795 1.01282051 1.07692308 1.14102564 1.20512821
 1.26923077 1.33333333 1.3974359  1.46153846 1.52564103 1.58974359
 1.65384615 1.71794872 1.78205128 1.84615385 1.91025641 1.97435897
 2.03846154 2.1025641  2.16666667 2.23076923 2.29487179 2.35897436
 2.42307692 2.48717949 2.55128205 2.61538462 2.67948718 2.74358974
 2.80769231 2.87179487 2.93589744 3.        ]
